In [448]:
import pandas as pd
import os
import time
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from dotenv import load_dotenv
load_dotenv()



False

In [449]:
folderFile = r"C:\Users\gabriel.sandres\OneDrive - Sicoob\Área de Trabalho\cod_fonte_registro\registro-chamados-codigo-fonte\planilha_registro.xlsx"

In [450]:
df = pd.read_excel(folderFile)


In [451]:
for i,l in df.iterrows():
    print(i,l[0],l[1],l[2],l[3],l[4])
    #cpf = l[0]
    #cpf = '05534899174'
    #cooperativa = '3179'

    #login = 'gabriel.sandres'
    #senha = 'C@tolica61136'
    print("Fim  loop")
 
    

0 055.348.991-74 3179 WP12345678 Pix Dúvida Técnica
Fim  loop
1 05534899174 3179 COB123456789 Cobraça Bancária Dúvida Técnica
Fim  loop


C:\Users\gabriel.sandres\AppData\Local\Temp\ipykernel_23516\4280248987.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(i,l[0],l[1],l[2],l[3],l[4])


In [452]:
cpf ='05534899174'
cooperativa = '3179'
categoria ='Pix'
subcategoria = "Api Sicoob"
servico = "Dúvida Negocial"
protocolo_plad = "WP12345"
descricao = "Chamado registrado via automação"

login = 'gabriel.sandres'
senha = 'C@tolica61136'

In [453]:
# Bloco 1 – Acessar a URL do Visão 360 (com webdriver-manager)
 
options = Options()
options.add_argument("--start-maximized")  # Abre o navegador em tela cheia
 
# Inicializa o driver automaticamente com webdriver-manager
service = ChromeService(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
 
# Acessa o Visão 360
url = "https://portal.sisbr.coop.br/auth/realms/sisbr/protocol/openid-connect/auth?&scope=openid&client_id=visao360-sisbr&response_type=code&redirect_uri=https://portal.sisbr.coop.br/visao360/consult"
driver.get(url)
 
# Aguarda o carregamento da página
import time
time.sleep(5)

In [454]:
# Bloco 2 – Realizar o login com os dados do .env
 
# Captura login e senha do .env
#usuario = os.getenv("LOGIN_USERNAME")
#senha = os.getenv("LOGIN_PASSWORD")

 
# Espera e preenche o campo de login
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "username"))).send_keys(login)
 
# Preenche o campo de senha
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "password"))).send_keys(senha)
 
# Clica no botão "Logar"
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "kc-login"))).click()
 
# Espera o QR Code aparecer (tempo ajustável, ex: 30s para você escanear e digitar o código)
print("Aguardando escaneamento do QR Code e inserção do código...")
WebDriverWait(driver, 60).until(lambda d: "visao360/consult" in d.current_url)
print("QR Code validado com sucesso!")

Aguardando escaneamento do QR Code e inserção do código...
QR Code validado com sucesso!


In [456]:
# Bloco 3 – Preencher CPF/CNPJ e clicar em "Consultar"
 
 # Aguarda o overlay sumir antes de prosseguir
WebDriverWait(driver, 20).until(
    EC.invisibility_of_element_located((By.CLASS_NAME, "ngx-spinner-overlay"))
)
 
# Espera o campo de documento estar presente e clica antes de digitar (boa prática com Angular)
campo_documento = WebDriverWait(driver, 15).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="app"]/section/sc-content/sc-consult/div/div[2]/div/sc-card-content/div/main/form/div/div[2]/sc-form-field/div/input'))
)                                        

campo_documento.click()
campo_documento.clear()
campo_documento.send_keys(cpf)  # 'cpf' já definido anteriormente
 
# Aguarda botão 'Consultar' ficar habilitado (classe muda ou botão fica clicável)
botao_consultar = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="app"]/section/sc-content/sc-consult/div/div[2]/div/sc-card-content/div/main/form/div/div[3]/sc-button/button'))
)

botao_consultar.click()
 
print("Documento preenchido e consulta realizada com sucesso.")




Documento preenchido e consulta realizada com sucesso.


In [457]:
# Bloco 4 – Clicar no botão "Abrir"
 
# Aguarda o botão "Abrir" ficar visível e clicável

botao_abrir = WebDriverWait(driver, 10).until(

    EC.element_to_be_clickable((By.XPATH, '//*[@id="app"]/section/sc-content/sc-consult/div/div[2]/div/sc-card-content/div/main/form/div/div[4]/sc-card/div/sc-card-content/div/div/div[2]/sc-button/button'))

)
 
# Clica no botão "Abrir"

botao_abrir.click()
 
print("Botão 'Abrir' clicado com sucesso.")

 

Botão 'Abrir' clicado com sucesso.


In [458]:
# Bloco 5 – Selecionar conta conforme a cooperativa
 
# Aguarda o <select> de contas estar disponível
select_element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "accounts"))
)
 
# Captura todas as opções da conta
options = select_element.find_elements(By.TAG_NAME, "option")
 
# Percorre as opções e seleciona a que contém a cooperativa
conta_encontrada = False
for option in options:
    if f"Coop: {cooperativa}" in option.text:
        option.click()
        conta_encontrada = True
        print(f"Conta da cooperativa {cooperativa} selecionada com sucesso.")
        break
 
# Se nenhuma conta compatível for encontrada
if not conta_encontrada:
    print(f"⚠️ Nenhuma conta com cooperativa {cooperativa} encontrada.")

Conta da cooperativa 3179 selecionada com sucesso.


In [459]:
# Bloco 6 – Selecionar o produto "Cobrança"
 
# XPath do elemento que representa o card do produto "Cobrança"
xpath_cobranca = '//*[@id="products"]/div[10]/sc-card/div/div/div/div'
 
# Aguarda o produto "Cobrança" ficar clicável
produto_cobranca = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, xpath_cobranca))
)
 
# Clica no produto "Cobrança"
produto_cobranca.click()
time.sleep(1.5)  # tempo curto para a classe atualizar visualmente
 
# Verifica se o produto foi selecionado (classe mudou para 'selected-product')
pai = produto_cobranca.find_element(By.XPATH, "..")  # sobe um nível na hierarquia
if "selected-product" in pai.get_attribute("class"):
    print("Produto 'Cobrança' selecionado com sucesso.")
else:
    print("⚠️ Produto 'Cobrança' não foi selecionado corretamente.")

⚠️ Produto 'Cobrança' não foi selecionado corretamente.


In [460]:
# Bloco 7 – Abrir o formulário (clicar no botão com ícone de Registro de chamado)
 
# Aguarda o botão de "Registro de chamado" estar visível e clicável
botao_formulario = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//button[@tooltip='Registro de chamado']"))
)
 
# Clica no botão
botao_formulario.click()
 
print("Formulário de chamado aberto com sucesso.")

Formulário de chamado aberto com sucesso.


In [461]:
# Bloco 8 – Preencher corretamente o campo "Tipo de Atendimento"
 
# Aguarda o campo de tipo de atendimento
campo_tipo_atendimento = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "serviceTypeId"))
)
 
# Usa ActionChains para clicar e digitar como um humano
actions = ActionChains(driver)
actions.click(campo_tipo_atendimento).pause(0.3)
 
# Digita cada letra de "Chat Receptivo" lentamente
for letra in "Chat Receptivo":
    actions.send_keys(letra).pause(0.1)
 
# Pressiona seta para baixo e enter para selecionar da lista
actions.send_keys(Keys.ARROW_DOWN).pause(0.3).send_keys(Keys.ENTER).perform()
 
# Aguarda e verifica se o campo ficou válido
time.sleep(1)
classe_atual = campo_tipo_atendimento.get_attribute("class")
if "ng-valid" in classe_atual:
    print("✅ Campo 'Tipo de Atendimento' preenchido com sucesso.")
else:
    print("⚠️ O campo foi preenchido, mas a aplicação ainda não o validou.")

✅ Campo 'Tipo de Atendimento' preenchido com sucesso.


In [462]:
# Bloco 9 – Preencher o campo "Categoria"
 
# Aguarda o campo de categoria ficar clicável

campo_categoria = WebDriverWait(driver, 10).until(

    EC.element_to_be_clickable((By.ID, "categoryId"))

)
 
# Usa ActionChains para simular digitação realista

actions = ActionChains(driver)

actions.click(campo_categoria).pause(0.3)
 
# Digita lentamente o nome da categoria (exemplo: "Cobrança")

for letra in categoria:

    actions.send_keys(letra).pause(0.1)
 
# Seleciona a opção com seta para baixo + enter

actions.send_keys(Keys.ARROW_DOWN).pause(0.3).send_keys(Keys.ENTER).perform()
 
# Aguarda e valida com base na classe

time.sleep(1)

classe_atual = campo_categoria.get_attribute("class")

if "ng-valid" in classe_atual:

    print("✅ Campo 'Categoria' preenchido e validado com sucesso.")

else:

    print("⚠️ O campo 'Categoria' foi preenchido, mas não foi validado pela aplicação.")

 

✅ Campo 'Categoria' preenchido e validado com sucesso.


In [464]:
# Bloco 10 – Preencher o campo "Subcategoria"
 
# Aguarda o campo ficar clicável

campo_subcategoria = WebDriverWait(driver, 10).until(

    EC.element_to_be_clickable((By.ID, "subCategoryId"))

)
 
# Usa ActionChains para simular digitação realista

actions = ActionChains(driver)

actions.click(campo_subcategoria).pause(0.3)
 
# Digita lentamente o valor da subcategoria (ex: variável vinda da planilha)

for letra in subcategoria:

    actions.send_keys(letra).pause(0.1)
 
# Seleciona da lista usando ARROW_DOWN + ENTER

actions.send_keys(Keys.ARROW_DOWN).pause(0.3).send_keys(Keys.ENTER).perform()
 
# Aguarda e verifica se o campo foi validado corretamente

time.sleep(7)

classe_atual = campo_subcategoria.get_attribute("class")

if "ng-valid" in classe_atual:

    print("✅ Campo 'Subcategoria' preenchido e validado com sucesso.")

else:

    print("⚠️ O campo 'Subcategoria' foi preenchido, mas não foi validado pela aplicação.")

 

✅ Campo 'Subcategoria' preenchido e validado com sucesso.


In [466]:
# Bloco 11 – Preencher o campo "Serviço"
 
# Aguarda o campo de serviço ficar clicável

campo_servico = WebDriverWait(driver, 20).until(

    EC.element_to_be_clickable((By.ID, "serviceId"))

)
 
# Usa ActionChains para digitação simulada

actions = ActionChains(driver)

actions.click(campo_servico).pause(0.3)
 
# Digita o conteúdo da variável `servico` letra por letra

for letra in servico:

    actions.send_keys(letra).pause(0.1)
 
# Simula seleção com seta para baixo + enter

actions.send_keys(Keys.ARROW_DOWN).pause(0.3).send_keys(Keys.ENTER).perform()
 
# Aguarda e valida o preenchimento

time.sleep(1)

classe_atual = campo_servico.get_attribute("class")

if "ng-valid" in classe_atual:

    print("✅ Campo 'Serviço' preenchido e validado com sucesso.")

else:

    print("⚠️ O campo 'Serviço' foi preenchido, mas não foi validado pela aplicação.")

 #Preencher o formulario - Subcategoria

✅ Campo 'Serviço' preenchido e validado com sucesso.


In [468]:
# Bloco 12 – Preencher o campo "Canal de autoatendimento"
 
# Aguarda o select ficar presente

select_element = WebDriverWait(driver, 10).until(

    EC.presence_of_element_located((By.ID, "Canal De Autoatendimento"))

)
 
# Usa Select do Selenium para selecionar pelo texto visível

select = Select(select_element)

select.select_by_visible_text("Não Se Aplica")
 
# Aguarda e valida se o campo foi aceito (classe ng-valid)

time.sleep(0.5)

classe_atual = select_element.get_attribute("class")

if "ng-valid" in classe_atual:

    print("✅ Campo 'Canal de autoatendimento' selecionado com sucesso.")

else:

    print("⚠️ O campo 'Canal de autoatendimento' foi preenchido, mas não foi validado pela aplicação.")

 

✅ Campo 'Canal de autoatendimento' selecionado com sucesso.


In [469]:
# Bloco 13 – Preencher o campo "Protocolo PLAD"
 
# Aguarda o campo estar clicável

campo_protocolo = WebDriverWait(driver, 10).until(

    EC.element_to_be_clickable((By.ID, "Protocolo Plad"))

)
 
# Clica e digita o valor da variável protocolo_plad

campo_protocolo.click()

campo_protocolo.clear()

campo_protocolo.send_keys(protocolo_plad)
 
# Aguarda e valida o preenchimento

time.sleep(1)

classe_atual = campo_protocolo.get_attribute("class")

if "ng-valid" in classe_atual:

    print("✅ Campo 'Protocolo PLAD' preenchido e validado com sucesso.")

else:

    print("⚠️ O campo 'Protocolo PLAD' foi preenchido, mas não foi validado pela aplicação.")

 

✅ Campo 'Protocolo PLAD' preenchido e validado com sucesso.


In [470]:
# Bloco 14 – Preencher o campo "Descrição"
 
# Verifica se a descrição é válida ou precisa usar fallback

descricao_final = descricao if descricao and len(descricao.strip()) >= 10 else "Chamado registrado automaticamente via RPA"
 
# Aguarda o textarea de descrição ficar clicável

campo_descricao = WebDriverWait(driver, 10).until(

    EC.element_to_be_clickable((By.ID, "description"))

)
 
# Clica, limpa e digita a descrição

campo_descricao.click()

campo_descricao.clear()

campo_descricao.send_keys(descricao_final)
 
# Aguarda e valida o preenchimento

time.sleep(1)

classe_atual = campo_descricao.get_attribute("class")

if "ng-valid" in classe_atual:

    print("✅ Campo 'Descrição' preenchido e validado com sucesso.")

else:

    print("⚠️ O campo 'Descrição' foi preenchido, mas não foi validado pela aplicação.")
 

✅ Campo 'Descrição' preenchido e validado com sucesso.


In [471]:
# Bloco 15 – Clicar no botão "Registrar"
 
# Aguarda o botão "Registrar" ficar clicável

botao_registrar = WebDriverWait(driver, 20).until(

    EC.element_to_be_clickable((By.XPATH, '//*[@id="actionbar hide"]/div/div[2]/form/div/div[20]/sc-button/button'))

)
 
# Clica no botão

botao_registrar.click()
 
print("✅ Primeiro clique no botão 'Registrar' realizado com sucesso.")

 

✅ Primeiro clique no botão 'Registrar' realizado com sucesso.


In [472]:
# Bloco 16 – Clicar no segundo botão "Registrar" (confirmação do modal)
 
# Aguarda o botão "Registrar" dentro do modal ficar clicável

botao_confirmar_registro = WebDriverWait(driver, 20).until(

    EC.element_to_be_clickable((By.XPATH, '//*[@id="modal"]/div/sc-modal-footer/div/div/div[2]/sc-button/button'))

)
 
# Clica no botão de confirmação

botao_confirmar_registro.click()
 
print("✅ Segundo clique no botão 'Registrar' (confirmação) realizado com sucesso.")

 

✅ Segundo clique no botão 'Registrar' (confirmação) realizado com sucesso.


In [473]:
# Aguarda o elemento com o protocolo ficar visível
elemento_protocolo = WebDriverWait(driver, 20).until(
    EC.visibility_of_element_located((By.XPATH, '//*[@id="actionbar hide"]/div/div[2]/form/div/div[2]/sc-card/div/sc-card-content/div/div/div[1]/h5'))
)

# Extrai o texto (número do protocolo)
numero_protocolo = elemento_protocolo.text.strip()
print(f"✅ Protocolo capturado: {numero_protocolo}")

# Carrega a planilha
df = pd.read_excel(folderFile)

# Atualiza o valor na linha correspondente (índice i dentro do loop for)
df.loc[i, 'Protocolo Visão'] = numero_protocolo

# Salva de volta
df.to_excel(folderFile, index=False)
print("✅ Protocolo colado na planilha com sucesso.")


✅ Protocolo capturado: SPI20250610003257
✅ Protocolo colado na planilha com sucesso.


C:\Users\gabriel.sandres\AppData\Local\Temp\ipykernel_23516\1482184037.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'SPI20250610003257' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'Protocolo Visão'] = numero_protocolo


In [474]:
# Bloco – Clicar em "Finalizar Atendimento"

try:

    # Aguarda o botão "Finalizar Atendimento" estar clicável

    btn_finalizar = WebDriverWait(driver, 20).until(

        EC.element_to_be_clickable((By.XPATH, '//*[@id="actionbar hide"]/div/div[2]/form/div/div[5]/sc-button/button'))

    )

    btn_finalizar.click()

    print("✅ Botão 'Finalizar Atendimento' clicado com sucesso.")

except Exception as e:

    print("❌ Erro ao clicar no botão 'Finalizar Atendimento':", e)

 

✅ Botão 'Finalizar Atendimento' clicado com sucesso.


In [475]:
# Bloco – Clicar em "Confirmar" para finalizar o atendimento

try:

    # Aguarda o botão "Confirmar" estar clicável

    btn_confirmar = WebDriverWait(driver, 20).until(

        EC.element_to_be_clickable((By.XPATH, '//*[@id="modal"]/div/main/div/div[4]/button'))

    )

    btn_confirmar.click()

    print("✅ Botão 'Confirmar' clicado com sucesso.")

except Exception as e:

    print("❌ Erro ao clicar no botão 'Confirmar':", e)

 

✅ Botão 'Confirmar' clicado com sucesso.


In [ ]:
#Clicar em "Confirmar"